FIRST PART

In [2]:
import requests
import pandas as pd
import bs4 
from bs4 import BeautifulSoup

In [3]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
data = BeautifulSoup(url,'lxml')

In [4]:
data_table = data.find('table', class_ = 'wikitable')
table_rows = data_table.find_all('tr')

n = []
for row in table_rows:
    info = row.text.split('\n')[1:-1] 
    n.append(info)

In [5]:
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(n[1:], columns = column_names)
df.head()


,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) ]
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [7]:
group = df.groupby('Postalcode')
grouped_neighborhoods = group['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
grouped_boroughs = group['Borough'].apply(lambda x: set(x).pop())
grouped_df = pd.DataFrame(list(zip(grouped_boroughs.index, grouped_boroughs, grouped_neighborhoods)))
grouped_df.columns = ['Postalcode', 'Borough', 'Neighborhood']

grouped_df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
grouped_df.shape

(103, 3)

In [9]:
coord_df = pd.read_csv('Geospatial_Coordinates.csv') 


In [10]:
df2 = grouped_df.join(coord_df.set_index('Postal Code'), on='Postalcode')

In [11]:
df2.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
df2.shape

(103, 5)

SECOND PART

In [13]:
pip install folium==0.1.5

Note: you may need to restart the kernel to use updated packages.


In [16]:
import folium
map = folium.Map(location=[43.6532,-79.3832], zoom_start=11)

for location in df2.itertuples(): 
    label = 'Postalcode: {};  Borough: {};  Neighborhood: {}'.format(location[1], location[2], location[3])
    #label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [location[-2], location[-1]],
        radius=1,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map) 
    folium.Circle(
        radius=500,
        popup=label,
        location=[location[-2], location[-1]],
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(map) 
    
map

AttributeError: module 'folium' has no attribute 'CircleMarker'

In [13]:

import matplotlib.pyplot as plt

X = df2[['Longitude', 'Latitude']]

scatter_plot = X.plot.scatter(x='Longitude', 
               y='Latitude',  
               fontsize = 16,
               figsize=(15, 10))

scatter_plot.set_yticklabels([])
scatter_plot.set_xticklabels([])
scatter_plot.set_xticks([])
scatter_plot.set_yticks([])
scatter_plot.spines['top'].set_color(None)
scatter_plot.spines['right'].set_color(None)
plt.xlabel('Longitude', fontsize=16)
plt.ylabel('Latitude', fontsize=16)
scatter_plot.set_title('Toronto Postalcodes Position', fontsize=16)

scatter_plot.set_yticklabels(labels = 'Latitude', fontdict={'fontsize': 20})
plt.show()

<Figure size 1500x1000 with 1 Axes>

In [24]:
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

In [25]:
CLIENT_ID = 'STMX202ELDKLXE2O5LOC4L0UDMVUFMFSOP3J5IAQ21BGFER1' 
CLIENT_SECRET = 'JDBNYKZWNVWYI45GYW343KB130YS31EG4TLQ34EI2THXXSAG' 
VERSION = '20180604'

In [26]:
scarborough_data = df2[df2['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [27]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [29]:
map2 = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)


for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map2

AttributeError: module 'folium' has no attribute 'CircleMarker'

In [30]:
neighborhood_latitude = scarborough_data.loc[3, 'Latitude'] 
neighborhood_longitude = scarborough_data.loc[3, 'Longitude'] 

neighborhood_name = scarborough_data.loc[3, 'Neighborhood']

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Woburn" are 43.7709921, -79.21691740000001.


In [32]:
LIMIT = 100 # foursquare
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [33]:
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']

In [35]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
import json 
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) 


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(15)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
5,DAVIDsTEA,Tea Room,43.776613,-79.258516
6,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
7,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
8,Hot Topic,Clothing Store,43.775450,-79.257929
9,Tim Hortons,Coffee Shop,43.769936,-79.260368


In [37]:
nearby_venues.shape[0] # get the number of venues found in this neibourhood

43

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
s_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


In [44]:

s_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Clarks Corners, Sullivan, Tam O'Shanter",10,10,10,10,10,10
"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",7,7,7,7,7,7
"East Birchmount Park, Ionview, Kennedy Park",5,5,5,5,5,5


In [47]:
print('There are {} uniques categories.'.format(len(s_venues['Venue Category'].unique())))

There are 52 uniques categories.


In [48]:
scarb_onehot = pd.get_dummies(s_venues[['Venue Category']], prefix="", prefix_sep="")


scarb_onehot['Neighborhood'] =s_venues['Neighborhood'] 


fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

NameError: name 'scarb_grouped' is not defined

In [53]:

scarb_onehot.shape

(86, 53)

In [54]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.0,0.000000,0.000,0.0,0.25,0.0,0.000000,...,0.0,0.25,0.0,0.25,0.0,0.000000,0.0,0.000,0.0,0.0
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000,0.0,0.333333,0.000,0.0,0.00,0.0,0.000000,...,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.000,0.0,0.0
2,"Birch Cliff, Cliffside West",0.0,0.000,0.0,0.000000,0.000,0.0,0.00,0.0,0.000000,...,0.0,0.00,0.0,0.25,0.0,0.000000,0.0,0.000,0.0,0.0
3,Cedarbrae,0.0,0.125,0.0,0.125000,0.125,0.0,0.00,0.0,0.000000,...,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.125,0.0,0.0
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000,0.0,0.222222,0.000,0.0,0.00,0.0,0.222222,...,0.0,0.00,0.0,0.00,0.0,0.111111,0.0,0.000,0.0,0.0


In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [57]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Sandwich Place,Breakfast Spot,Lounge,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Bakery,Playground,Park,Vietnamese Restaurant,Chinese Restaurant,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Golf Course,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Hakka Restaurant,Bakery,Fried Chicken Joint,Lounge,Bank,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Auto Garage,Cosmetics Shop
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Park,Vietnamese Restaurant,Coffee Shop,General Entertainment


In [58]:
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(16)

k = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


kmeans = KMeans(n_clusters=k, random_state=0).fit(scarb_grouped_clustering)


kmeans.labels_[:10]

array([0, 0, 0, 0, 0, 0, 3, 0, 0, 0])

In [59]:
scarb_merged = scarb_data


scarb_merged['Cluster Labels'] = kmeans.labels_

scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarb_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Grocery Store,Golf Course,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Golf Course,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Electronics Store,Intersection,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Gym,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Hakka Restaurant,Bakery,Fried Chicken Joint,Lounge,Bank,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Auto Garage,Cosmetics Shop


In [60]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)


x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighborhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

AttributeError: module 'folium' has no attribute 'Popup'

In [61]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Grocery Store,Golf Course,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,Scarborough,0,Golf Course,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,Scarborough,0,Electronics Store,Intersection,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Fried Chicken Joint
3,Scarborough,0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Gym,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,Scarborough,0,Hakka Restaurant,Bakery,Fried Chicken Joint,Lounge,Bank,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Auto Garage,Cosmetics Shop
5,Scarborough,0,Health & Beauty Service,Spa,Playground,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
7,Scarborough,0,Bakery,Bus Line,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Park,Vietnamese Restaurant,Coffee Shop,General Entertainment
8,Scarborough,0,American Restaurant,Motel,Gym,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
9,Scarborough,0,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Golf Course,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
11,Scarborough,0,Auto Garage,Smoke Shop,Shopping Mall,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant


In [62]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Scarborough,1,Chinese Restaurant,Fast Food Restaurant,Gym,Sandwich Place,Cosmetics Shop,Grocery Store,Pharmacy,Pizza Place,Bubble Tea Shop,Breakfast Spot


In [63]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,2,Indian Restaurant,Vietnamese Restaurant,Pet Store,Thrift / Vintage Store,Latin American Restaurant,Chinese Restaurant,Bar,Breakfast Spot,Athletics & Sports,General Entertainment


In [64]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,3,Discount Store,Chinese Restaurant,Department Store,Coffee Shop,Thrift / Vintage Store,Grocery Store,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant


In [65]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,4,Pizza Place,Chinese Restaurant,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Bank,Italian Restaurant,Pharmacy,Noodle House,Vietnamese Restaurant
